Now let's just see how using two jits witht an in intrisinc breaks things:

In [1]:
import os
os.environ['NUMBA_DEBUG'] = '1'

In [2]:
import llvmlite

import numba

llvm_type = llvmlite.ir.IntType(64)


class TestStructType(numba.types.Type):
    def __init__(self):
        super().__init__(name="TestStruct")


test_struct_type = TestStructType()


@numba.extending.register_model(TestStructType)
class TestStructModel(numba.extending.models.PrimitiveModel):
    def __init__(self, dmm, fe_type):
        super().__init__(dmm, fe_type, llvmlite.ir.PointerType(llvm_type))


@numba.extending.intrinsic
def create_test_struct(typingctx, int_t):

    sig = test_struct_type(int_t)

    def codegen(context, builder, sig, args):
        a = builder.alloca(llvm_type)
        builder.store(value=args[0], ptr=a)
        return a

    return sig, codegen


@numba.extending.intrinsic
def convert_test_struct(typingctx, test_struct_t):

    sig = numba.types.int64(test_struct_t)

    def codegen(context, builder, sig, args):
        return builder.load(args[0])

    return sig, codegen

/Users/saul/p/numba-xnd/numba/numba/__init__.py:152: UserWarning: SVML was not found/could not be loaded.
  warnings.warn("SVML was not found/could not be loaded.")


In [3]:

@numba.njit
def inner_fn_works():
    a = create_test_struct(10)
    print("inner_fn_works", convert_test_struct(a))
    return a


@numba.njit
def outer_fn_works():
    print("outer_fn_works", convert_test_struct(inner_fn_works()))

outer_fn_works()




--------------------------FUNCTION OPTIMIZED DUMP nrt---------------------------
; ModuleID = 'nrt'
source_filename = "<string>"
target datalayout = "e-m:o-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-apple-darwin17.7.0"

@.const.picklebuf.4675369160 = internal constant { i8*, i32 } { i8* getelementptr inbounds ([101 x i8], [101 x i8]* @.const.pickledata.4675369160, i32 0, i32 0), i32 101 }
@.const.pickledata.4675369160 = internal constant [101 x i8] c"\80\04\95Z\00\00\00\00\00\00\00\8C\08builtins\94\8C\0CRuntimeError\94\93\94\8C6numba jitted function aborted due to unresolved symbol\94\85\94\86\94."

define i64 @nrt_atomic_add(i64* %.1) {
.3:
  %.4 = atomicrmw add i64* %.1, i64 1 monotonic
  %.5 = add i64 %.4, 1
  ret i64 %.5
}

define i64 @nrt_atomic_sub(i64* %.1) {
.3:
  %.4 = atomicrmw sub i64* %.1, i64 1 monotonic
  %.5 = sub i64 %.4, 1
  ret i64 %.5
}

define i32 @nrt_atomic_cas(i64* %.1, i64 %.2, i64 %.3, i64* %.4) {
.6:
  %.7 = cmpxchg i64* %.1, i64 %.2, i64 %.3 mon

In [5]:
@numba.njit
def inner_fn_broken():
    a = create_test_struct(10)
    return a


@numba.njit
def outer_fn_broken():
    print("outer_fn_broken", convert_test_struct(inner_fn_broken()))



outer_fn_broken()

----------------------------IR DUMP: outer_fn_broken----------------------------
label 0:
    $0.1 = global(print: <built-in function print>) ['$0.1']
    $const0.2 = const(str, outer_fn_broken)  ['$const0.2']
    $0.3 = global(convert_test_struct: <intrinsic convert_test_struct>) ['$0.3']
    $0.4 = global(inner_fn_broken: CPUDispatcher(<function inner_fn_broken at 0x116eaef28>)) ['$0.4']
    $0.5 = call $0.4(func=$0.4, args=[], kws=(), vararg=None) ['$0.4', '$0.5']
    del $0.4                                 []
    $0.6 = call $0.3($0.5, func=$0.3, args=[Var($0.5, <ipython-input-5-285f2053bd07> (9))], kws=(), vararg=None) ['$0.3', '$0.5', '$0.6']
    del $0.5                                 []
    del $0.3                                 []
    $0.7 = call $0.1($const0.2, $0.6, func=$0.1, args=[Var($const0.2, <ipython-input-5-285f2053bd07> (9)), Var($0.6, <ipython-input-5-285f2053bd07> (9))], kws=(), vararg=None) ['$0.1', '$0.6', '$0.7', '$const0.2']
    del $const0.2               